# Load the AVISO Finite-Size Lyapanov Exponent

### Imports

In [1]:

import sys
import os
import glob

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.gridspec import GridSpec
import matplotlib.colors as mcolors
import cmocean.cm as cmo
from cmocean.tools import lighten
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# statistics
import scipy as sci

# regridding package
import xesmf as xe

# copernicusmarine
import copernicusmarine

# print versions of packages
print("python version =",sys.version[:5])
print("numpy version =", np.__version__)
print("pandas version =", pd.__version__)
print("scipy version =", sci.__version__)
print("xarray version =", xr.__version__)
print("xesmf version =", xe.__version__)
print("cartopy version =", sys.modules[ccrs.__package__].__version__)
print("matplotlib version =", sys.modules[plt.__package__].__version__)
print("cmocean version =", sys.modules[cmo.__package__].__version__)


wrkdir="/g/data/es60/pjb581/SPC"
os.chdir(wrkdir)
os.getcwd()

from dask.diagnostics import ProgressBar
ProgressBar().register()


python version = 3.10.
numpy version = 1.26.4
pandas version = 2.2.3
scipy version = 1.15.3
xarray version = 2024.1.1
xesmf version = 0.8.10
cartopy version = 0.24.1
matplotlib version = 3.10.0
cmocean version = v3.0.3


### Load GLORYS data

In [5]:
%%time

DSphy = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_phy_my_0.083deg_P1M-m")
DSphy = DSphy.chunk({"time":1, "depth":-1, "latitude":-1, "longitude":216})
# Print memory usage in bytes (NumPy array)
print("Memory of each 4D dataarray (Gbytes):", DSphy.so.nbytes*1e-9)

DSphy_late = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_phy_myint_0.083deg_P1M-m")
DSphy_late = DSphy_late.chunk({"time":1, "depth":-1, "latitude":-1, "longitude":216})
print("Memory of each 4D dataarray (Gbytes):", DSphy_late.so.nbytes*1e-9)

DSphy_late



INFO - 2025-07-06T23:33:02Z - Selected dataset version: "202311"
INFO - 2025-07-06T23:33:02Z - Selected dataset part: "default"


Memory of each 4D dataarray (Gbytes): 603.091008


INFO - 2025-07-06T23:33:45Z - Selected dataset version: "202311"
INFO - 2025-07-06T23:33:45Z - Selected dataset part: "default"


Memory of each 4D dataarray (Gbytes): 81.11750400000001
CPU times: user 32.1 s, sys: 1.97 s, total: 34.1 s
Wall time: 1min 1s


<xarray.Dataset>
Dimensions:    (depth: 50, latitude: 2041, longitude: 4320, time: 46)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * time       (time) datetime64[ns] 2021-07-01 2021-08-01 ... 2025-04-01
Data variables:
    bottomT    (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    mlotst     (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    siconc     (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    sithick    (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    so         (time, depth, latitude, longitude) float32 dask.array<chunksize=(1, 50, 2041, 216), meta=np.ndarray>
    thetao     (time, depth, latitude, longitude) float32 dask.array<chunksize=(1, 50, 2041, 216), meta=np.ndarray>
    uo         (time, depth, latitude, longitude) float32 dask.array<chunksize=(1, 50, 2041, 216), meta=np.ndarray>
    usi        (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    vo         (time, depth, latitude, longitude) float32 dask.array<chunksize=(1, 50, 2041, 216), meta=np.ndarray>
    vsi        (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    zos        (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    contact:      servicedesk.cmems@mercator-ocean.eu
    credit:       E.U. Copernicus Marine Service Information (CMEMS)
    references:   http://marine.copernicus.eu
    source:       MERCATOR GLORYS12V1
    producer:     CMEMS - Global Monitoring and Forecasting Centre
    institution:  Mercator Ocean
    title:        Monthly mean fields for product GLOBAL_REANALYSIS_PHY_001_030

### save only the surface velocities

In [8]:
%%time

DSphy = DSphy.isel(depth=0)
DSphy_late = DSphy_late.isel(depth=0)

DSphy


CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 6.2 μs


<xarray.Dataset>
Dimensions:    (latitude: 2041, longitude: 4320, time: 342)
Coordinates:
    depth      float32 0.494
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * time       (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2021-06-01
Data variables:
    bottomT    (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    mlotst     (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    siconc     (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    sithick    (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    so         (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    thetao     (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    uo         (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    usi        (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    vo         (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    vsi        (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    zos        (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    contact:      servicedesk.cmems@mercator-ocean.eu
    credit:       E.U. Copernicus Marine Service Information (CMEMS)
    references:   http://marine.copernicus.eu
    source:       MERCATOR GLORYS12V1
    producer:     CMEMS - Global Monitoring and Forecasting Centre
    institution:  Mercator Ocean
    title:        Monthly mean fields for product GLOBAL_REANALYSIS_PHY_001_030

In [9]:
%%time

DSphy = DSphy.drop_vars(['zos', 'vsi', 'usi', 'thetao', 'siconc', 'sithick', 'so', 'bottomT'])
DSphy_late = DSphy_late.drop_vars(['zos', 'vsi', 'usi', 'thetao', 'siconc', 'sithick', 'so', 'bottomT'])

DSphy


CPU times: user 814 μs, sys: 11 μs, total: 825 μs
Wall time: 847 μs


<xarray.Dataset>
Dimensions:    (latitude: 2041, longitude: 4320, time: 342)
Coordinates:
    depth      float32 0.494
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * time       (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2021-06-01
Data variables:
    mlotst     (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    uo         (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    vo         (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    contact:      servicedesk.cmems@mercator-ocean.eu
    credit:       E.U. Copernicus Marine Service Information (CMEMS)
    references:   http://marine.copernicus.eu
    source:       MERCATOR GLORYS12V1
    producer:     CMEMS - Global Monitoring and Forecasting Centre
    institution:  Mercator Ocean
    title:        Monthly mean fields for product GLOBAL_REANALYSIS_PHY_001_030

In [10]:
%%time

DSphy = xr.concat([DSphy, DSphy_late], dim='time')
del DSphy_late
print("Memory of each 4D dataarray (Gbytes):", DSphy.uo.nbytes*1e-9)
DSphy


Memory of each 4D dataarray (Gbytes): 13.68417024
CPU times: user 19.6 ms, sys: 9.61 ms, total: 29.2 ms
Wall time: 27.3 ms


<xarray.Dataset>
Dimensions:    (latitude: 2041, longitude: 4320, time: 388)
Coordinates:
    depth      float32 0.494
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * time       (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2025-04-01
Data variables:
    mlotst     (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    uo         (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
    vo         (time, latitude, longitude) float32 dask.array<chunksize=(1, 2041, 216), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    contact:      servicedesk.cmems@mercator-ocean.eu
    credit:       E.U. Copernicus Marine Service Information (CMEMS)
    references:   http://marine.copernicus.eu
    source:       MERCATOR GLORYS12V1
    producer:     CMEMS - Global Monitoring and Forecasting Centre
    institution:  Mercator Ocean
    title:        Monthly mean fields for product GLOBAL_REANALYSIS_PHY_001_030

### take a look

In [ ]:
%%time

DSphy.compute()

In [ ]:
%%time

plt.figure()
plt.pcolormesh(DSphy.uo.isel(time=0))
plt.colorbar()


### Calculate dx and dy

In [9]:
%%time

lat = DSphy.latitude.values  # shape (2041,)
lon = DSphy.longitude.values  # shape (4320,)

# Earth's radius in meters
R = 6371000  

# Convert degrees to radians
lat_rad = np.deg2rad(lat)
lon_rad = np.deg2rad(lon)

# Latitude spacing (dy): constant in degrees, but actual distance varies with latitude
dlat = np.gradient(lat_rad)  # shape (2041,)
dy = dlat * R  # meters between latitude lines

# Longitude spacing (dx): depends on latitude due to spherical convergence
dlon = np.gradient(lon_rad)  # shape (4320,)
dx = np.outer(np.cos(lat_rad), dlon) * R  # shape (2041, 4320)


CPU times: user 23.4 ms, sys: 34.1 ms, total: 57.5 ms
Wall time: 54.8 ms


### Calculate Okubo-Weiss

In [ ]:
%%time

u_surf = DSphy.uo.isel(time=0)
v_surf = DSphy.vo.isel(time=0)

# Get gradients (2D)
du_dy = np.gradient(u_surf, axis=0) / dy[:, np.newaxis]
du_dx = np.gradient(u_surf, axis=1) / dx
dv_dy = np.gradient(v_surf, axis=0) / dy[:, np.newaxis]
dv_dx = np.gradient(v_surf, axis=1) / dx

strain = np.sqrt((dudx - dvdy)**2 + (dvdx + dudy)**2)
vorticity = dvdx - dudy
OW = strain**2 - vorticity**2


### take a look

In [ ]:
plt.figure()
plt.pcolormesh(OW)
plt.colorbar()

### regrid to coarse resolution

### saver to netcdf